In [1]:
import numpy as np
import pandas as pd
import random as rand

import requests
import json
import re
from json import loads

In [3]:
bus_station = pd.read_csv('서울시버스정류소좌표데이터(2022.08.24).csv', encoding='euc-kr')
bus_station = bus_station.sample(n=100)
bus_station

,NODE_ID,ARS-ID,정류소명,좌표X,좌표Y
1507,104000112,5205,건국대학교앞,127.074928,37.538577
1247,103000279,4722,무학여고,127.034051,37.558285
4592,111000335,12430,진관중고교.약수사앞,126.923140,37.643400
923,102000274,3732,이촌동두산위브트레지움,126.966001,37.521546
3956,110000184,11284,석계역2번출구,127.067004,37.615194
...,...,...,...,...,...
1073,103000130,4231,사근고개,127.044515,37.561232
10466,123000584,24233,한성백제박물관,127.119327,37.515347
1093,103900213,4256,성수사거리,127.062464,37.542522
5186,112900118,13855,민속식당,126.929665,37.567986


In [4]:
# Starting point는 0 부여
loc0_name = '강남역'
loc0_X = '127.028361548'
loc0_Y = '37.496486063'

In [5]:
# 목적지는 1부터 네이밍;; 목적지 개수는 (num-1) 개

num = 20 # 총 노드 개수

for i in range(1,num): # 1~(n-1)
    globals()[f'loc{i}_name'] = bus_station.iloc[i,2]
    globals()[f'loc{i}_X'] = bus_station.iloc[i,3]
    globals()[f'loc{i}_Y'] = bus_station.iloc[i,4]

In [6]:
print(loc1_name, loc1_X, loc1_Y)

무학여고 127.0340505738 37.5582850004


In [7]:
url = "https://apis.openapi.sk.com/tmap/matrix?version=1"

payload = {
    "origins": [
        {
            "lon": str(globals()[f'loc{i}_X']),
            "lat": str(globals()[f'loc{i}_Y']),
        } for i in range(num)
    ],
    "destinations": [
        {
            "lon": str(globals()[f'loc{i}_X']),
            "lat": str(globals()[f'loc{i}_Y']),
        } for i in range(num)
    ],
    "transportMode": "car",
    "metric": "Recommendation"
}
headers = {
    "Accept": "application/json",
    "appKey": "l7xx4d23b06a733d4f2e9c5365ca49c3431a",
    "Content-Type": "application/json"
}

response = requests.post(url, json=payload, headers=headers)

In [8]:
jsonObj = json.loads(response.text)
jsonObj

{'meta': {'status': 'Ok',
  'message': 'Ok',
  'moduleVersion': '',
  'mapVersion': '20220920',
  'elapsedTime': 0,
  'realTimeTrafficDateTime': '',
  'patternTrafficDate': '',
  'customizationDateTime': '',
  'algorithm': '',
  'metric': 'Recommendation',
  'requestId': '1c0aa0a9-1cd1-43be-b42b-03f1c36a5fd5',
  'gasStationPriceDateTime': '',
  'transportMode': 'Car',
  'avoid': []},
 'origins': [{'link': {'linkId': '0', 'direction': 'TwoWay'},
   'poiId': '',
   'name': '',
   'coordinate': {'latitude': 37.496486063, 'longitude': 127.028361548},
   'rpFlag': ''},
  {'link': {'linkId': '0', 'direction': 'TwoWay'},
   'poiId': '',
   'name': '',
   'coordinate': {'latitude': 37.5582850004, 'longitude': 127.0340505738},
   'rpFlag': ''},
  {'link': {'linkId': '0', 'direction': 'TwoWay'},
   'poiId': '',
   'name': '',
   'coordinate': {'latitude': 37.6434001879, 'longitude': 126.9231400464},
   'rpFlag': ''},
  {'link': {'linkId': '0', 'direction': 'TwoWay'},
   'poiId': '',
   'name': '

In [9]:
jsonObj['matrixRoutes'][0]

{'status': 'Ok',
 'originIndex': 0,
 'destinationIndex': 0,
 'cost': 0,
 'duration': 0,
 'distance': 0.0,
 'toll': False}

In [10]:
duration_lst = []
distance_lst = []
for i in range(num**2):
    duration_lst.append(jsonObj['matrixRoutes'][i]['duration'])
    distance_lst.append(jsonObj['matrixRoutes'][i]['distance'])
duration = np.array(duration_lst).reshape(num,num)
distance = np.array(distance_lst).reshape(num,num)

In [11]:
print(duration.shape)
duration

(20, 20)


array([[   0, 1913, 5010, 1793, 3043, 3432, 2923, 4331, 4229, 2383, 2051,
        2694, 2735, 3459, 2147, 3042, 3613, 2577, 2304, 3603],
       [2259,    0, 3767, 1899, 1728, 2117, 2756, 3679, 3577, 3927,  138,
        3214,  825, 2144, 1396, 3148, 1858, 2683, 2121, 3708],
       [4734, 3390,    0, 3489, 3070, 3041, 2169, 1250, 1516, 4520, 3318,
        3051, 3521, 2836, 3322, 2567, 2923, 2923, 4306, 3214],
       [2060, 1637, 3439,    0, 2937, 3327, 1638, 2984, 2390, 2311, 1775,
        1265, 2630, 3353, 2413, 1471, 3508, 1005, 2894, 1903],
       [3493, 1998, 3774, 3311,    0,  776, 3715, 3807, 3706, 5160, 1596,
        3646, 1515,  658, 1842, 3380, 1406, 4094, 2406, 3862],
       [4035, 2541, 3737, 3853,  661,    0, 4246, 4339, 4237, 5471, 2415,
        4178, 2058,  332, 2284, 3911, 2225, 4637, 2449, 4394],
       [3078, 2232, 2311, 1523, 2363, 2897,    0, 1490, 1388, 3403, 2160,
        2015, 2383, 2779, 1153, 1142, 2068,  957, 4183, 2509],
       [4132, 2520, 1042, 2980, 2348, 288

In [13]:
def distance_matrix():
    response = requests.post(url, json=payload, headers=headers)
    jsonObj = json.loads(response.text)

    duration_lst = []
    distance_lst = []
    for i in range(num**2):
        duration_lst.append(jsonObj['matrixRoutes'][i]['duration'])
        distance_lst.append(jsonObj['matrixRoutes'][i]['distance'])
    
    duration = np.array(duration_lst).reshape(num,num)
    distance = np.array(distance_lst).reshape(num,num)

    return duration

distance_matrix()

array([[   0, 1986, 5128, 1826, 3046, 3611, 3020, 4426, 4130, 2618, 2105,
        3288, 2611, 3345, 2137, 3294, 3530, 2891, 2463, 4081],
       [1950,    0, 3863, 1703, 1561, 2126, 2985, 3215, 3532, 3939,  157,
        3410,  840, 1860, 1463, 3171, 1953, 2768, 1973, 4014],
       [5102, 3855,    0, 3855, 3202, 3369, 2437, 1432, 1835, 5088, 3794,
        3219, 3827, 2873, 3464, 2709, 3415, 3088, 4245, 2774],
       [1968, 1849, 3450,    0, 2910, 3475, 1708, 3019, 2941, 2539, 1968,
        1548, 2474, 3209, 2322, 1616, 3394, 1213, 2953, 2153],
       [2954, 1735, 3910, 2854,    0,  811, 3865, 3417, 3734, 5089, 1854,
        3876, 1321,  545, 1826, 3785, 1459, 3919, 2375, 4423],
       [3547, 2328, 3574, 3447,  610,    0, 4140, 3692, 4009, 5682, 2447,
        4151, 1914,  400, 2446, 4060, 2027, 4511, 2694, 4699],
       [2480, 1982, 2218, 1314, 2261, 2825,    0, 1468, 1172, 3371, 2110,
        1976, 2173, 2560, 1027, 1108, 2061,  843, 3741, 2414],
       [4084, 2919,  987, 2782, 2679, 324